In [ ]:


from citibike.citibike_utils import get_trip_duration_min
from pyspark.sql.functions import *
pipeline_id = dbutils.widgets.get('pipeline_id')
run_id = dbutils.widgets.get('run_id')
task_id = dbutils.widgets.get('task_id')
processed_timestamp = dbutils.widgets.get('processed_timestamp')
catalog = dbutils.widgets.get('catalog')
catalog = 'circuitbike_dev'
# Read the data from the bronze layer
df_silver = spark.read.table(f'{catalog}.bronze.jc_citibike')

NameError: name 'spark' is not defined

In [ ]:
df = df_silver.get_trip_duration_min(spark, df_silver, 'started_at', 'ended_at')

In [ ]:
df_transform = df_silver.select('ride_id', split(col('started_at'),' ')[0].cast('date').alias('trip_start_date'), col('started_at').cast('timestamp'), col('ended_at').cast('timestamp')
                                , 'start_station_name', 'end_station_name', (((unix_timestamp(col('ended_at').cast('timestamp')) - unix_timestamp(col('started_at').cast('timestamp'))) / 60)).cast("decimal(10,2)").alias('trip_duration_mins'))
#display(df_transform)

NameError: name 'df_silver' is not defined

In [ ]:
df_transform = df_transform.withColumn('metadata', 
                                 create_map(lit('pipeline_id'), lit(pipeline_id),
                                            lit('run_id'), lit(run_id),
                                            lit('task_id'), lit(task_id),
                                            lit('processed_timestamp'), lit(processed_timestamp)))

In [ ]:
df_transform.write.mode('overwrite').option('overwriteSchema', 'True').saveAsTable(f'{catalog}.silver.jc_citibike')